<a href="https://colab.research.google.com/github/gabibu/aiforhealthcare/blob/main/advanced_models_colab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydicom

     |████████████████████████████████| 2.0 MB 4.2 MB/s 


In [27]:
from torch.utils.data import Dataset, WeightedRandomSampler
from skimage import io, transform
import os
import pandas as pd
from PIL import Image
import random
import numpy as np
from torchvision import transforms, datasets
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from imageio import imread
import torch
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import cm

import pydicom
from abc import ABC, abstractmethod
from torchvision.transforms import RandomHorizontalFlip, RandomVerticalFlip, RandomRotation
from sklearn.model_selection import train_test_split

In [28]:
def mask2rle(img, width, height):
    rle = []
    lastColor = 0;
    currentPixel = 0;
    runStart = -1;
    runLength = 0;

    for x in range(width):
        for y in range(height):
            currentColor = img[x][y]
            if currentColor != lastColor:
                if currentColor == 255:
                    runStart = currentPixel;
                    runLength = 1;
                else:
                    rle.append(str(runStart));
                    rle.append(str(runLength));
                    runStart = -1;
                    runLength = 0;
                    currentPixel = 0;
            elif runStart > -1:
                runLength += 1
            lastColor = currentColor;
            currentPixel+=1;

    return " ".join(rle)

def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]

    return mask.reshape(width, height)


In [29]:
print('__pyTorch VERSION:', torch.__version__)
use_cuda = torch.cuda.is_available()
print('use_cuda = {0}'.format(use_cuda))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

DATA_FOLDER ='data'
device

__pyTorch VERSION: 1.11.0+cu113
use_cuda = True


'cuda'

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
!ls "/content/drive/My Drive/medical"


aug.weights		simple_preprocessing.weights  weighted_2.weights
dicom-images-train.zip	train-rle.csv		      weighted_sampling.weights


In [32]:
!cp "/content/drive/My Drive/medical/dicom-images-train.zip" "dicom-images-train.zip"


In [33]:
!cp '/content/drive/My Drive/medical/train-rle.csv' "train-rle.csv"


In [34]:
!ls

data  dicom-images-train.zip  drive  sample_data  train-rle.csv


In [ ]:
!unzip  -q dicom-images-train.zip -d data 


In [35]:
SEED = 999

def fixSeed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

fixSeed(SEED)
SEED

999

In [ ]:
# df = pd.read_csv('train-rle.csv', header=None, index_col=0)
# train_fns = sorted(glob.glob('data/dicom-images-train/*/*/*.dcm'))
# len(train_fns)
# df_full = pd.read_csv('train-rle.csv', index_col='ImageId')
# df_full.sample(1)

In [36]:
!ls data

dicom-images-train


In [37]:
train_imgs_paths = sorted(glob.glob('data/dicom-images-train/*/*/*.dcm'))
print("Train images size {}".format(len(train_imgs_paths)))

# test_imgs_paths = sorted(glob('../input/siim-acr-pneumothorax-segmentation-data/dicom-images-test/*/*/*.dcm'))
# print("Test images -", len(test_imgs_paths))
# file_paths = train_imgs_paths + test_imgs_paths

Train images size 10712


In [38]:
# data_df = pd.read_csv('train-rle.csv')
# data_df.rename(columns={" EncodedPixels" : "EncodedPixels"}, inplace=True) # a typo in the csv
# data_df.head()

rles_df = pd.read_csv('train-rle.csv')
rles_df = rles_df.rename(columns={' EncodedPixels':'EncodedPixels'})
rles_df['EncodedPixels'] = rles_df['EncodedPixels'].apply(lambda x: x.strip())

# Create a dictionary for images with masks
rles_df = rles_df[rles_df['EncodedPixels'] !='-1'].groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
print(len(rles_df))

masks = {}
for index, row in rles_df.iterrows():
    masks[row['ImageId']] = row['EncodedPixels']
print(len(masks))

2379
2379


In [45]:
dataset_rows = []
for image_path in train_imgs_paths:
  id = image_path.split('/')[-1][:-4]
  
  mask = masks[id] if id in masks else None
  dataset_rows.append((id, mask, image_path))

dataset_df = pd.DataFrame(dataset_rows, columns = ['image_id', 'mask', 'image_path'])



,image_id,mask,image_path
9104,1.2.276.0.7230010.3.1.4.8323329.4329.151787518...,[191086 2 1017 8 1012 12 1009 14 1008 15 1007 ...,data/dicom-images-train/1.2.276.0.7230010.3.1....
1436,1.2.276.0.7230010.3.1.4.8323329.11314.15178752...,[338146 2 1020 3 1019 5 1017 7 1016 7 1015 8 1...,data/dicom-images-train/1.2.276.0.7230010.3.1....
10277,1.2.276.0.7230010.3.1.4.8323329.540.1517875163...,None,data/dicom-images-train/1.2.276.0.7230010.3.1....


In [47]:
X_train_df, X_val_df = train_test_split(dataset_df, test_size=0.1, random_state=SEED)
X_train_df.shape, X_val_df.shape

((9640, 3), (1072, 3))

In [ ]:
for image in 

In [ ]:
class ImagesDataset(Dataset):
    def __init__(self, images, masks, preprocesing = None):
        self.images = images
        self.masks = masks
        self.preprocesing = preprocesing
        
      
    def __len__(self):
        return self.masks.shape[0]

    def __getitem__(self, index):
        
        y = self.masks[index]
        y = np.where(y == True, 1, 0)
        
        image  = X_train[index]
        
        if self.preprocesing is not None:
            for pre_processor in self.preprocesing:
                
                image = pre_processor.preprocess(image)
                
        
        image = image.reshape(1, image.shape[0], image.shape[1])
        y = y.reshape(1, y.shape[0], y.shape[1])
        
        return image, y